<br>
<center>
<img src="https://raw.githubusercontent.com/elaynelemos/prediction-of-orders-dmc/main/assets/img/univasf-logo.png" width=200>
<h3>
    UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO
    <br>COLEGIADO DE ENGENHARIA DE COMPUTAÇÃO
</h3>

<h3>Orientador</h3>
<span>Prof. Dr. Rosalvo Ferreira de Oliveira Neto</span>

<h3>Discentes</h3>
<span>Anísio Pereira Batista Filho
<br>Edjair Aguiar Gomes Filho
<br>Elayne Rute Lessa Lemos</span>
</center>
<br><br>

## Regressão com Redes Neurais

Projeto em: [https://github.com/anisiobfilho/regression_neuralnetworks_univasf](https://github.com/anisiobfilho/regression_neuralnetworks_univasf)

### Importação das bibliotecas

In [1]:
#OS
import os
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Matplotlib
import matplotlib.pyplot as plt

### Importação das bases de dados

In [2]:
repo_url = 'https://raw.githubusercontent.com/anisiobfilho/regression_neuralnetworks_univasf/main'

In [3]:
ic_house_pred_train = pd.read_csv(f'{repo_url}/data/ic_house_pred_train.csv')
ic_house_pred_train.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
73,89,50,C (all),105.0,8470,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,IDOTRR,Feedr,Feedr,1Fam,1.5Fin,3,2,1915,1982,Hip,CompShg,Plywood,Plywood,None,0.0,Fa,Fa,CBlock,TA,Fa,No,Unf,0,Unf,0,1013,1013,GasA,...,N,SBrkr,1013,0,513,1526,0,0,1,0,2,1,Fa,6,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,0,156,0,0,0,NaN,MnPrv,NaN,0,10,2009,ConLD,Abnorml,85000
717,607,20,RL,82.0,12464,Pave,NaN,IR2,Low,AllPub,Corner,Mod,CollgCr,Norm,Norm,1Fam,1Story,5,5,1996,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,PConc,Gd,TA,No,GLQ,732,Unf,0,308,1040,GasA,...,Y,SBrkr,1040,0,0,1040,1,0,1,0,3,1,Gd,6,Typ,0,NaN,Detchd,2000.0,Unf,2,576,TA,TA,Y,168,0,0,0,0,0,NaN,GdPrv,NaN,0,11,2009,WD,Normal,152000
596,690,120,RL,61.0,7577,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,TwnhsE,1Story,6,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,Stone,256.0,Gd,TA,PConc,Gd,TA,Av,ALQ,20,Unf,0,1342,1362,GasA,...,Y,SBrkr,1362,0,0,1362,0,0,2,0,2,1,Gd,6,Typ,1,Gd,Attchd,2005.0,RFn,2,460,TA,TA,Y,192,28,0,0,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal,194700
707,18,90,RL,72.0,10791,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,4,5,1967,1967,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,...,Y,SBrkr,1296,0,0,1296,0,0,2,0,2,2,TA,6,Typ,0,NaN,CarPort,1967.0,Unf,2,516,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,Shed,500,10,2006,WD,Normal,90000
750,688,160,FV,NaN,5105,Pave,NaN,IR2,Lvl,AllPub,FR2,Gtl,Somerst,Norm,Norm,TwnhsE,2Story,7,5,2004,2004,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,239,Unf,0,312,551,GasA,...,Y,SBrkr,551,551,0,1102,0,0,2,1,2,1,Gd,4,Typ,0,NaN,Detchd,2004.0,Unf,2,480,TA,TA,Y,0,60,0,0,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal,148800


In [4]:
ic_house_pred_test = pd.read_csv(f'{repo_url}/data/ic_house_pred_test.csv')
ic_house_pred_test.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
281,1450,180,RM,21.0,1533,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,SFoyer,5,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,Gd,TA,Av,GLQ,553,Unf,0,77,630,GasA,...,Y,SBrkr,630,0,0,630,1,0,1,0,1,1,Ex,3,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD,Abnorml,92000
347,180,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,...,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal,100000
377,110,20,RL,105.0,11751,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1977,1977,Hip,CompShg,Plywood,Plywood,BrkFace,480.0,TA,TA,CBlock,Gd,TA,No,BLQ,705,Unf,0,1139,1844,GasA,...,Y,SBrkr,1844,0,0,1844,0,0,2,0,3,1,TA,7,Typ,1,TA,Attchd,1977.0,RFn,2,546,TA,TA,Y,0,122,0,0,0,0,NaN,MnPrv,NaN,0,1,2010,COD,Normal,190000
334,579,160,FV,34.0,3604,Pave,Pave,Reg,Lvl,AllPub,Corner,Gtl,Somerst,Norm,Norm,TwnhsE,2Story,7,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,689,689,GasA,...,Y,SBrkr,703,689,0,1392,0,0,2,0,2,1,Gd,5,Typ,0,NaN,Detchd,2007.0,Unf,2,540,TA,TA,Y,0,102,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml,146000
425,292,190,RL,55.0,5687,Pave,Grvl,Reg,Bnk,AllPub,Inside,Gtl,SWISU,Norm,Norm,2fmCon,2Story,5,6,1912,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Fa,PConc,TA,Fa,No,Rec,210,Unf,0,570,780,GasA,...,N,SBrkr,936,780,0,1716,1,0,2,0,6,1,Fa,9,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,184,0,0,0,0,NaN,NaN,NaN,0,3,2008,WD,Normal,135900


### Separação da base em features e variável alvo

In [5]:
ic_house_pred_train_X = ic_house_pred_train.drop(columns='SalePrice')
ic_house_pred_train_X.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
592,285,120,RL,50.0,8012,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,TwnhsE,1Story,6,5,1992,1992,Gable,CompShg,Plywood,ImStucc,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,430,Unf,0,1145,1575,GasA,Gd,Y,SBrkr,1575,0,0,1575,1,0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2,529,TA,TA,Y,0,0,52,0,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal
998,277,20,RL,129.0,9196,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Ex,TA,No,Unf,0,Unf,0,1560,1560,GasA,Ex,Y,SBrkr,1560,0,0,1560,0,0,2,0,3,1,Gd,7,Typ,0,NaN,Attchd,2003.0,Fin,2,573,TA,TA,Y,100,150,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
1001,386,120,RL,43.0,3182,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,BrkFace,16.0,Gd,TA,PConc,Gd,TA,No,GLQ,24,Unf,0,1232,1256,GasA,Ex,Y,SBrkr,1269,0,0,1269,0,0,2,0,2,1,Gd,6,Typ,1,TA,Attchd,2004.0,Fin,2,430,TA,TA,Y,146,20,0,0,144,0,NaN,NaN,NaN,0,4,2010,WD,Normal
333,146,160,RM,24.0,2522,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,Twnhs,2Story,6,5,2004,2006,Gable,CompShg,VinylSd,VinylSd,Stone,50.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,970,970,GasA,Ex,Y,SBrkr,970,739,0,1709,0,0,2,0,3,1,Gd,7,Maj1,0,NaN,Detchd,2004.0,Unf,2,380,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal
853,710,20,RL,NaN,7162,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,7,1966,1966,Gable,CompShg,HdBoard,HdBoard,BrkCmn,41.0,TA,TA,PConc,TA,TA,No,Unf,0,Unf,0,876,876,GasA,TA,Y,SBrkr,904,0,0,904,0,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1966.0,Unf,1,408,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,12,2008,WD,Abnorml


In [6]:
ic_house_pred_train_y = pd.DataFrame(ic_house_pred_train['SalePrice'], columns=['SalePrice'])
ic_house_pred_train_y.sample(5)

,SalePrice
793,294000
918,138000
318,149900
282,191000
822,176500


In [7]:
ic_house_pred_test_X = ic_house_pred_test.drop(columns='SalePrice')
ic_house_pred_test_X.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
160,33,20,RL,85.0,11049,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,1Fam,1Story,8,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1234,1234,GasA,Ex,Y,SBrkr,1234,0,0,1234,0,0,2,0,3,1,Gd,7,Typ,0,NaN,Attchd,2007.0,RFn,2,484,TA,TA,Y,0,30,0,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal
138,1034,20,RL,NaN,8125,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,Stone,295.0,Gd,TA,PConc,Gd,TA,No,GLQ,986,Unf,0,668,1654,GasA,Ex,Y,SBrkr,1654,0,0,1654,1,0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,2002.0,Unf,3,900,TA,TA,Y,0,136,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Normal
318,1311,20,RL,100.0,17500,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,PosA,Norm,1Fam,1Story,7,8,1959,2002,Gable,CompShg,BrkFace,HdBoard,None,0.0,Gd,Gd,PConc,Gd,TA,Av,GLQ,1406,Unf,0,496,1902,GasA,TA,Y,SBrkr,1902,0,0,1902,1,0,2,0,3,1,Ex,7,Typ,2,TA,Attchd,1959.0,Fin,2,567,TA,TA,Y,0,207,162,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
373,535,60,RL,74.0,9056,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,8,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Ex,Gd,Av,Unf,0,Unf,0,707,707,GasA,Ex,Y,SBrkr,707,707,0,1414,0,0,2,1,3,1,Gd,6,Typ,1,Gd,Attchd,2004.0,Fin,2,403,TA,TA,Y,100,35,0,0,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal
39,260,20,RM,70.0,12702,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,5,1956,1956,Gable,CompShg,BrkFace,BrkFace,None,0.0,TA,TA,PConc,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,Gd,Y,FuseA,882,0,0,882,0,0,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1956.0,Unf,1,308,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [8]:
ic_house_pred_test_y = pd.DataFrame(ic_house_pred_test['SalePrice'], columns=['SalePrice'])
ic_house_pred_test_y.sample(5)

,SalePrice
170,360000
156,108959
0,154500
287,89471
107,275000


### Eliminar a variável irrelevante PID (Nominal): Parcel identification number

In [9]:
ic_house_pred_train_X.drop(columns='Id', inplace=True)
ic_house_pred_train_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
370,60,RL,72.0,7226,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,798,798,GasA,Ex,Y,SBrkr,798,842,0,1640,0,0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2003.0,RFn,2,595,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
462,40,RM,40.0,5400,Pave,Pave,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1Story,6,7,1926,2004,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,BrkTil,TA,TA,Mn,LwQ,370,Unf,0,779,1149,GasA,Gd,Y,FuseA,1149,467,0,1616,0,0,2,0,3,1,Gd,5,Typ,0,NaN,Detchd,1926.0,Unf,1,216,TA,TA,Y,0,0,183,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Normal
451,120,RL,32.0,10846,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Veenker,Norm,Norm,TwnhsE,1Story,8,5,1993,1993,Gable,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,1619,Unf,0,100,1719,GasA,Ex,Y,SBrkr,1719,0,0,1719,2,0,1,1,1,1,Gd,6,Typ,2,Gd,Attchd,1993.0,Fin,2,473,TA,TA,Y,122,30,0,0,0,0,NaN,NaN,NaN,0,5,2008,Con,Normal
1012,90,RL,NaN,10624,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,Duplex,1Story,5,4,1964,1964,Gable,CompShg,HdBoard,HdBoard,BrkFace,84.0,TA,TA,CBlock,TA,TA,No,GLQ,40,Rec,264,1424,1728,GasA,TA,Y,SBrkr,1728,0,0,1728,0,1,2,0,6,2,TA,10,Typ,0,NaN,Detchd,2002.0,Unf,1,352,TA,TA,Y,155,0,0,0,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
447,60,RL,107.0,13891,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,9,5,2008,2009,Hip,CompShg,VinylSd,VinylSd,Stone,424.0,Ex,TA,PConc,Ex,TA,Gd,Unf,0,Unf,0,1734,1734,GasA,Ex,Y,SBrkr,1734,1088,0,2822,0,0,3,1,4,1,Ex,12,Typ,1,Gd,BuiltIn,2009.0,RFn,3,1020,TA,TA,Y,52,170,0,0,192,0,NaN,NaN,NaN,0,1,2009,New,Partial


In [10]:
ic_house_pred_test_X.drop(columns='Id', inplace=True)
ic_house_pred_test_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
383,80,RL,NaN,14112,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,SLvl,5,7,1964,1964,Hip,CompShg,Wd Sdng,HdBoard,BrkFace,86.0,TA,TA,PConc,TA,TA,Av,GLQ,1014,Unf,0,138,1152,GasA,TA,Y,SBrkr,1152,0,0,1152,1,0,1,0,3,1,TA,6,Typ,1,Gd,Attchd,1964.0,RFn,2,484,TA,TA,Y,227,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
280,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0,Unf,0,936,936,GasA,Ex,Y,SBrkr,936,840,0,1776,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002.0,RFn,2,474,TA,TA,Y,144,96,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Normal
93,60,RL,86.0,10380,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1986,1987,Gable,CompShg,Plywood,Plywood,BrkFace,172.0,Gd,TA,CBlock,TA,TA,Gd,LwQ,28,ALQ,1474,0,1502,GasA,Ex,Y,SBrkr,1553,1177,0,2730,1,0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1987.0,Fin,2,576,TA,TA,Y,201,96,0,0,0,0,NaN,MnPrv,NaN,0,8,2007,WD,Normal
84,60,RL,74.0,7844,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,2Story,6,7,1978,1978,Hip,CompShg,HdBoard,HdBoard,BrkFace,203.0,TA,TA,CBlock,TA,TA,No,ALQ,209,Unf,0,463,672,GasA,TA,Y,SBrkr,672,728,0,1400,0,0,1,1,3,1,TA,6,Typ,1,TA,Attchd,1978.0,Fin,2,440,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal
72,20,RL,77.0,8593,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,6,1957,1957,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,288,Unf,0,619,907,GasA,Ex,Y,SBrkr,907,0,0,907,0,0,1,0,3,1,TA,5,Typ,0,NaN,Detchd,1964.0,Unf,1,352,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal


### Tratamento dos Missing Values (Apenas colocar para a média)

In [11]:
# replace_missing_by_fixed_value() é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def replace_missing_by_fixed_value(df, value, features):
    for column in features:
        df[column].fillna(value, inplace=True)

    return df

In [12]:
# replace_missing_by_mean() é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# Livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def replace_missing_by_mean(df, numeric_features):
    for column in numeric_features:
        average = df[column].mean(axis=0)
        df[column].fillna(average, inplace=True)
    
    return df

In [13]:
print(ic_house_pred_train_X.columns.values)

['MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'Utilities' 'LotConfig' 'LandSlope'
 'Neighborhood' 'Condition1' 'Condition2' 'BldgType' 'HouseStyle'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle'
 'RoofMatl' 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea'
 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType'
 'GarageYrBlt' 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual'
 'GarageCond' 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'PoolQC' 'Fence' 'MiscFeature'
 'MiscVal' 'MoSold' 'YrSold' 'SaleType' '

In [14]:

# FEATURES:
nominal_features = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 
                    'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                    'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 
                    'MiscFeature', 'SaleType', 'SaleCondition']

ordinal_features = ['LotShape', 'Utilities',  'LandSlope', 'OverallQual', 'OverallCond', 
                    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                    'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 
                    'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 
                    'PavedDrive', 'PoolQC', 'Fence']

discrete_features = ['YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 
                     'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 
                     'GarageYrBlt', 'GarageCars', 'MoSold', 'YrSold']

continuous_features = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 
                       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
                       'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
                       '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']


print("LENGHT:\n\tNominal: %d\n\tOrdinal: %d\n\tDiscrete: %d\n\tContinuous: %d" % (len(nominal_features), len(ordinal_features), len(discrete_features), len(continuous_features)))

LENGHT:
	Nominal: 23
	Ordinal: 23
	Discrete: 14
	Continuous: 19


#### ic_house_pred_train

In [15]:
ic_house_pred_train_X.columns[ic_house_pred_train_X.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [16]:
ic_house_pred_train_X = replace_missing_by_fixed_value(ic_house_pred_train_X, 'ausente', nominal_features)
ic_house_pred_train_X = replace_missing_by_fixed_value(ic_house_pred_train_X, 'ausente', ordinal_features)

ic_house_pred_train_X = replace_missing_by_mean(ic_house_pred_train_X, discrete_features)
ic_house_pred_train_X = replace_missing_by_mean(ic_house_pred_train_X, continuous_features)


In [17]:
ic_house_pred_train_X.columns[ic_house_pred_train_X.isnull().any()]

Index([], dtype='object')

In [18]:
ic_house_pred_train_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
672,50,RM,70.375,6240,Pave,ausente,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Fin,5,4,1936,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,Gd,TA,No,Unf,0,Unf,0,896,896,GasA,Gd,Y,FuseA,896,448,0,1344,0,0,1,0,3,1,TA,7,Typ,0,ausente,Detchd,1936.000000,Unf,1,240,Fa,TA,Y,200,114,0,0,0,0,ausente,ausente,ausente,0,4,2006,WD,Normal
717,20,RL,82.000,12464,Pave,ausente,IR2,Low,AllPub,Corner,Mod,CollgCr,Norm,Norm,1Fam,1Story,5,5,1996,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,PConc,Gd,TA,No,GLQ,732,Unf,0,308,1040,GasA,Gd,Y,SBrkr,1040,0,0,1040,1,0,1,0,3,1,Gd,6,Typ,0,ausente,Detchd,2000.000000,Unf,2,576,TA,TA,Y,168,0,0,0,0,0,ausente,GdPrv,ausente,0,11,2009,WD,Normal
871,60,RL,95.000,11787,Pave,ausente,IR1,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,594.0,Gd,TA,PConc,Gd,TA,No,GLQ,719,Unf,0,660,1379,GasA,Ex,Y,SBrkr,1383,1015,0,2398,1,0,2,1,3,1,Gd,8,Typ,1,TA,Attchd,1996.000000,Fin,3,834,TA,TA,Y,239,60,0,0,0,0,ausente,ausente,ausente,0,8,2007,WD,Normal
73,50,C (all),105.000,8470,Pave,ausente,IR1,Lvl,AllPub,Corner,Gtl,IDOTRR,Feedr,Feedr,1Fam,1.5Fin,3,2,1915,1982,Hip,CompShg,Plywood,Plywood,None,0.0,Fa,Fa,CBlock,TA,Fa,No,Unf,0,Unf,0,1013,1013,GasA,TA,N,SBrkr,1013,0,513,1526,0,0,1,0,2,1,Fa,6,Typ,0,ausente,ausente,1978.676653,ausente,0,0,ausente,ausente,N,0,0,156,0,0,0,ausente,MnPrv,ausente,0,10,2009,ConLD,Abnorml
984,20,RL,70.000,7420,Pave,ausente,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1962,1962,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,417,Unf,0,640,1057,GasA,TA,Y,SBrkr,1057,0,0,1057,0,0,1,0,3,1,TA,6,Typ,0,ausente,Detchd,1977.000000,Fin,2,576,TA,TA,Y,0,0,0,0,0,0,ausente,ausente,ausente,0,4,2007,WD,Normal


#### ic_house_pred_test

In [19]:
ic_house_pred_test_X.columns[ic_house_pred_test_X.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [20]:
ic_house_pred_test_X = replace_missing_by_fixed_value(ic_house_pred_test_X, 'ausente', nominal_features)
ic_house_pred_test_X = replace_missing_by_fixed_value(ic_house_pred_test_X, 'ausente', ordinal_features)

ic_house_pred_test_X = replace_missing_by_mean(ic_house_pred_test_X, discrete_features)
ic_house_pred_test_X = replace_missing_by_mean(ic_house_pred_test_X, continuous_features)


In [21]:
ic_house_pred_test_X.columns[ic_house_pred_test_X.isnull().any()]

Index([], dtype='object')

In [22]:
ic_house_pred_test_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
434,20,RM,49.0,5820,Pave,ausente,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,3,8,1955,2005,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,ALQ,256,Unf,0,906,1162,GasA,Ex,Y,SBrkr,1163,0,0,1163,1,0,1,0,3,1,TA,6,Typ,0,ausente,Attchd,1955.0,Unf,1,220,Fa,TA,Y,142,98,0,0,0,0,ausente,ausente,ausente,0,7,2006,WD,Normal
262,20,RL,75.0,10650,Pave,ausente,Reg,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,1Fam,1Story,5,6,1976,1976,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,Gd,Av,LwQ,182,ALQ,712,0,894,GasA,TA,Y,SBrkr,894,0,0,894,1,0,1,0,3,1,TA,5,Typ,0,ausente,Attchd,1976.0,Unf,1,308,TA,TA,Y,365,0,0,0,0,0,ausente,MnPrv,ausente,0,2,2010,WD,Normal
113,70,RM,51.0,6120,Pave,ausente,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,2Story,5,8,1920,2004,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,Mn,Unf,0,Unf,0,939,939,GasA,Ex,Y,SBrkr,939,574,0,1513,0,0,1,1,4,1,TA,8,Typ,0,ausente,Detchd,1933.0,Unf,1,180,Fa,Fa,N,24,0,150,0,0,0,ausente,ausente,ausente,0,5,2007,WD,Normal
280,60,RL,65.0,14006,Pave,ausente,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,ausente,Unf,0,Unf,0,936,936,GasA,Ex,Y,SBrkr,936,840,0,1776,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002.0,RFn,2,474,TA,TA,Y,144,96,0,0,0,0,ausente,ausente,ausente,0,2,2006,WD,Normal
98,20,RL,73.0,8990,Pave,ausente,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Mn,Unf,0,Unf,0,1498,1498,GasA,Ex,Y,SBrkr,1498,0,0,1498,0,0,2,0,2,1,Gd,5,Typ,0,ausente,Attchd,2006.0,RFn,2,675,TA,TA,Y,351,33,0,0,0,0,ausente,ausente,ausente,0,4,2006,New,Partial


### Criação de variáveis Dummies

In [23]:
ic_house_pred_train_X = pd.get_dummies(ic_house_pred_train_X, prefix_sep ='_', drop_first = True)
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,GarageType_ausente,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_ausente,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageQual_ausente,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_ausente,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_ausente,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_ausente,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MiscFeature_ausente,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1017,20,78.000,9317,6,5,2006,2006,0.0,24,0,1290,1314,1314,0,0,1314,0,0,2,0,3,1,6,1,2006.0,2,440,0,22,0,0,0,0,0,3,2007,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
230,20,70.375,6173,5,6,1967,1967,75.0,599,0,277,876,902,0,0,902,0,0,1,0,3,1,6,0,1967.0,1,288,0,0,0,0,0,0,0,8,2007,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
573,20,114.000,14803,10,5,2007,2008,816.0,1636,0,442,2078,2084,0,0,2084,1,0,2,0,2,1,7,1,2007.0,3,1220,188,45,0,0,0,0,0,6,2008,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
937,20,69.000,9920,5,6,1969,1969,0.0,523,0,448,971,971,0,0,971,0,0,1,1,3,1,5,1,1969.0,1,300,0,0,0,0,0,0,0,5,2006,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
948,30,52.000,6240,4,5,1925,1950,0.0,152,0,628,780,848,0,360,1208,0,0,1,0,2,1,5,0,1962.0,2,539,0,23,112,0,0,0,0,1,2009,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0


In [24]:
ic_house_pred_test_X = pd.get_dummies(ic_house_pred_test_X, prefix_sep ='_', drop_first = True)
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,FireplaceQu_TA,FireplaceQu_ausente,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_ausente,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_ausente,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageQual_ausente,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_ausente,PavedDrive_P,PavedDrive_Y,PoolQC_Gd,PoolQC_ausente,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_ausente,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_ausente,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
164,50,80.0,8480,5,5,1947,1950,0.0,442,0,390,832,832,384,0,1216,0,0,1,0,2,1,6,0,1947.0,1,336,158,0,102,0,0,0,0,10,2008,0,0,1,0,...,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
410,60,107.0,10186,7,5,1992,1992,0.0,674,0,76,750,1061,862,0,1923,1,0,2,1,3,1,8,1,1992.0,2,564,240,39,0,0,0,0,0,6,2010,0,0,1,0,...,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
103,45,51.0,6120,7,8,1929,2001,0.0,0,0,832,832,854,0,0,854,0,0,1,0,2,1,5,0,1991.0,2,576,48,112,0,0,0,0,0,7,2007,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
197,70,50.0,8500,4,4,1920,1950,0.0,0,0,649,649,649,668,0,1317,0,0,1,0,3,1,6,0,1920.0,1,250,0,54,172,0,0,0,0,7,2008,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
175,50,80.0,17120,4,4,1959,1959,0.0,0,0,0,0,1120,468,0,1588,0,0,2,0,4,1,7,1,1991.0,2,680,0,59,0,0,0,0,0,7,2008,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0


In [25]:
# O código abaixo é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

# Verificar as variáveis que estão no conjunto de treinamento e não estão no conjunto de teste;
# Verificar as variáveis que estão no conjunto de teste e não estão no conjunto de treinamento.

var_TRN = list(ic_house_pred_train_X.columns)
var_TST = list(ic_house_pred_test_X.columns)

In_TRN_Not_TST = np.setdiff1d(var_TRN, var_TST)
In_TST_Not_TRN = np.setdiff1d(var_TST, var_TRN)

if len(In_TRN_Not_TST)>0:
    print('Variáveis que estão apenas no Treinamento:\n')
    print(In_TRN_Not_TST)
else:
    print('Todas as variáveis do Treinamento estão no Teste!')
    
if len(In_TST_Not_TRN)>0:
    print('\nVariáveis que estão apenas no Teste:\n')
    print(In_TST_Not_TRN)
else:
    print('\nTodas as variáveis do Teste estão no Treinamento!')

Variáveis que estão apenas no Treinamento:

['Condition1_RRNn' 'Condition2_Feedr' 'Condition2_PosN' 'Condition2_RRAe'
 'Condition2_RRAn' 'Electrical_FuseP' 'Electrical_ausente' 'ExterCond_Po'
 'Exterior1st_AsphShn' 'Exterior1st_BrkComm' 'Exterior1st_CBlock'
 'Exterior2nd_CBlock' 'Exterior2nd_Other' 'Foundation_Wood'
 'Functional_Sev' 'GarageCond_Fa' 'GarageQual_Fa' 'HeatingQC_Po'
 'Heating_GasA' 'Heating_OthW' 'MiscFeature_TenC' 'PoolQC_Fa'
 'RoofMatl_CompShg' 'RoofMatl_Metal' 'RoofMatl_Roll' 'RoofStyle_Shed'
 'SaleCondition_AdjLand' 'SaleType_CWD' 'SaleType_Con' 'Utilities_NoSeWa']

Variáveis que estão apenas no Teste:

['Condition2_PosA' 'Condition2_RRNn' 'Electrical_Mix'
 'Exterior1st_ImStucc' 'RoofMatl_Membran']


In [26]:
# Adicionando as colunas faltantes em ambos os Dataframes

for column_name in In_TRN_Not_TST:
    ic_house_pred_test_X[column_name] = 0
    
for column_name in In_TST_Not_TRN:
    ic_house_pred_train_X[column_name] = 0

In [27]:
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageQual_ausente,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_ausente,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_ausente,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_ausente,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MiscFeature_ausente,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition2_PosA,Condition2_RRNn,Electrical_Mix,Exterior1st_ImStucc,RoofMatl_Membran
307,60,92.000,9920,7,5,1996,1997,0.0,862,0,255,1117,1127,886,0,2013,1,0,2,1,3,1,8,1,1997.0,2,455,180,130,0,0,0,0,0,6,2007,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
630,60,70.375,12205,6,8,1966,2007,157.0,568,0,264,832,976,1111,0,2087,0,0,2,1,5,1,9,0,1966.0,2,444,133,168,0,0,0,0,0,7,2007,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
333,160,24.000,2522,6,5,2004,2006,50.0,0,0,970,970,970,739,0,1709,0,0,2,0,3,1,7,0,2004.0,2,380,0,40,0,0,0,0,0,4,2006,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
151,20,72.000,9000,6,5,1956,1956,74.0,616,0,580,1196,1196,0,0,1196,1,0,1,0,2,1,6,1,1956.0,1,297,0,44,0,0,0,0,0,5,2008,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
857,20,85.000,11900,7,5,1977,1977,209.0,822,0,564,1386,1411,0,0,1411,0,0,2,0,3,1,6,1,1977.0,2,544,192,0,0,0,0,0,0,4,2009,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0


In [28]:
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition1_RRNn,Condition2_Feedr,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Electrical_FuseP,Electrical_ausente,ExterCond_Po,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_CBlock,Exterior2nd_CBlock,Exterior2nd_Other,Foundation_Wood,Functional_Sev,GarageCond_Fa,GarageQual_Fa,HeatingQC_Po,Heating_GasA,Heating_OthW,MiscFeature_TenC,PoolQC_Fa,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofStyle_Shed,SaleCondition_AdjLand,SaleType_CWD,SaleType_Con,Utilities_NoSeWa
206,60,80.0,9828,8,5,1995,1995,0.0,584,0,544,1128,1142,878,0,2020,0,0,2,1,3,1,8,1,1995.0,2,466,0,155,0,0,0,0,0,6,2009,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
189,160,21.0,2001,4,5,1970,1970,80.0,0,0,546,546,546,546,0,1092,0,0,1,1,3,1,6,0,1970.0,1,286,0,0,0,0,0,0,0,1,2007,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
366,20,78.0,10140,6,6,1974,1999,99.0,663,377,0,1040,1309,0,0,1309,1,0,1,1,3,1,5,1,1974.0,2,484,265,0,0,0,0,648,0,1,2006,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114,60,79.0,9245,7,5,2006,2006,0.0,0,0,939,939,939,858,0,1797,0,0,2,1,3,1,8,0,2006.0,2,639,144,53,0,0,0,0,0,4,2007,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
191,30,60.0,5400,5,6,1921,1968,0.0,0,0,1073,1073,1073,0,0,1073,0,0,1,0,2,1,4,0,1968.0,1,326,0,0,112,0,0,0,0,12,2006,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Transformação da variável alvo com a função logaritmo natural np.log(x)

In [29]:
ic_house_pred_train_y_log = pd.DataFrame(np.log(ic_house_pred_train_y['SalePrice']), columns=['SalePrice'])
ic_house_pred_train_y_log.sample(5)

,SalePrice
291,12.700769
780,11.626254
435,12.072541
723,11.941456
196,12.077671


In [30]:
ic_house_pred_test_y_log = pd.DataFrame(np.log(ic_house_pred_test_y['SalePrice']), columns=['SalePrice'])
ic_house_pred_test_y_log.sample(5)

,SalePrice
101,12.323411
207,11.842229
391,11.652687
435,12.235389
426,12.319401


### Normalização das variáveis

In [31]:
# uniformização de valores numéricos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

ic_house_pred_train_X_norm = scaler.fit_transform(ic_house_pred_train_X)
ic_house_pred_test_X_norm = scaler.fit_transform(ic_house_pred_test_X)

ic_house_pred_train_y_norm = scaler.fit_transform(ic_house_pred_train_y)
ic_house_pred_train_y_log_norm = scaler.fit_transform(ic_house_pred_train_y_log)
ic_house_pred_test_y_norm = scaler.fit_transform(ic_house_pred_test_y)
ic_house_pred_test_y_log_norm = scaler.fit_transform(ic_house_pred_test_y_log)

In [32]:
ic_house_pred_train_X[ic_house_pred_train_X.columns] = ic_house_pred_train_X_norm
ic_house_pred_train_y[ic_house_pred_train_y.columns] = ic_house_pred_train_y_norm
ic_house_pred_train_y_log[ic_house_pred_train_y_log.columns] = ic_house_pred_train_y_log_norm
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageQual_ausente,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageCond_ausente,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_ausente,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_ausente,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MiscFeature_ausente,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition2_PosA,Condition2_RRNn,Electrical_Mix,Exterior1st_ImStucc,RoofMatl_Membran
947,0.235294,0.304795,0.057903,0.888889,0.50,0.949275,0.900000,0.481858,0.180014,0.0,0.238014,0.257283,0.284075,0.530751,0.0,0.439714,0.333333,0.0,0.666667,0.5,0.375,0.5,0.666667,0.666667,0.936364,0.75,0.511989,0.466744,0.000000,0.0,0.0,0.0,0.0,0.0,0.181818,0.50,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.000000,0.219178,0.040665,0.777778,0.50,0.971014,0.933333,0.297533,0.000000,0.0,0.679795,0.259902,0.287747,0.000000,0.0,0.236247,0.000000,0.0,0.666667,0.0,0.375,0.5,0.416667,0.333333,0.963636,0.75,0.581805,0.168028,0.082267,0.0,0.0,0.0,0.0,0.0,1.000000,0.25,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.235294,0.169092,0.071257,0.777778,0.50,0.913043,0.800000,0.530479,0.138377,0.0,0.215325,0.210147,0.223956,0.552058,0.0,0.398644,0.333333,0.0,0.666667,0.5,0.375,0.5,0.416667,0.333333,0.890909,0.75,0.753879,0.000000,0.230347,0.0,0.0,0.0,0.0,0.0,0.363636,0.75,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
616,0.176471,0.133562,0.020033,0.555556,0.75,0.347826,0.800000,0.000000,0.000000,0.0,0.385702,0.147463,0.173015,0.053269,0.0,0.162773,0.000000,0.0,0.333333,0.0,0.500,0.5,0.416667,0.000000,0.715242,0.00,0.000000,0.000000,0.179159,0.0,0.0,0.0,0.0,0.0,0.727273,0.50,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
820,0.235294,0.212329,0.040688,0.666667,0.50,0.760870,0.450000,0.216981,0.069454,0.0,0.328767,0.189853,0.188619,0.419370,0.0,0.318011,0.000000,0.0,0.666667,0.5,0.500,0.5,0.500000,0.333333,0.700000,0.50,0.356135,0.336056,0.213894,0.0,0.0,0.0,0.0,0.0,0.181818,0.50,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
ic_house_pred_test_X[ic_house_pred_test_X.columns] = ic_house_pred_test_X_norm
ic_house_pred_test_y[ic_house_pred_test_y.columns] = ic_house_pred_test_y_norm
ic_house_pred_test_y_log[ic_house_pred_test_y_log.columns] = ic_house_pred_test_y_log_norm
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition1_RRNn,Condition2_Feedr,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Electrical_FuseP,Electrical_ausente,ExterCond_Po,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_CBlock,Exterior2nd_CBlock,Exterior2nd_Other,Foundation_Wood,Functional_Sev,GarageCond_Fa,GarageQual_Fa,HeatingQC_Po,Heating_GasA,Heating_OthW,MiscFeature_TenC,PoolQC_Fa,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofStyle_Shed,SaleCondition_AdjLand,SaleType_CWD,SaleType_Con,Utilities_NoSeWa
110,0.058824,0.196078,0.066825,0.000,0.142857,0.455224,0.000000,0.0,0.000000,0.00000,0.122620,0.082346,0.132341,0.0,0.0,0.083420,0.0,0.0,0.333333,0.0,0.166667,0.0,0.111111,0.333333,0.681046,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.75,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170,0.000000,0.450980,0.157168,0.875,0.428571,0.955224,0.900000,0.0,0.582267,0.00000,0.288899,0.591391,0.605459,0.0,0.0,0.381648,0.5,0.0,0.666667,0.0,0.500000,0.0,0.555556,1.000000,0.930000,0.75,0.509353,0.402321,0.348606,0.00000,0.0,0.0,0.0,0.0,0.909091,0.00,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
292,0.000000,0.320261,0.099740,0.375,0.571429,0.611940,0.116667,0.0,0.421389,0.00000,0.182072,0.409857,0.344913,0.0,0.0,0.217414,0.5,0.0,0.333333,0.0,0.500000,0.0,0.222222,0.000000,0.470000,0.25,0.211511,0.483559,0.000000,0.00000,0.0,0.0,0.0,0.0,0.454545,1.00,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.000000,0.315798,0.202209,0.750,0.571429,0.753731,0.433333,0.0,0.532450,0.27137,0.000000,0.488147,1.000000,0.0,0.0,0.630344,0.5,0.0,0.666667,0.0,0.333333,0.0,0.777778,0.333333,0.660000,0.50,0.478417,0.000000,0.143426,0.54717,0.0,0.0,0.0,0.0,0.363636,0.50,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,0.000000,0.267974,1.000000,0.625,0.428571,0.746269,0.416667,0.0,0.299360,0.00000,0.407803,0.478166,0.435484,0.0,0.0,0.274505,0.5,0.0,0.666667,0.0,0.333333,0.0,0.222222,0.666667,0.650000,0.50,0.414388,0.386847,0.107570,0.00000,0.0,0.0,0.0,0.0,1.000000,0.00,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Exportar duas bases de dados:

In [36]:
import os
path = 'data/preprocessed'
if not os.path.exists(path):
    os.mkdir(path)

ic_house_pred_train_X.to_csv(f'{path}/ic_house_pred_train_X.csv', index=False)
ic_house_pred_train_y.to_csv(f'{path}/ic_house_pred_train_y.csv', index=False)
ic_house_pred_train_y_log.to_csv(f'{path}/ic_house_pred_train_y_log.csv', index=False)

ic_house_pred_test_X.to_csv(f'{path}/ic_house_pred_test_X.csv', index=False)
ic_house_pred_test_y.to_csv(f'{path}/ic_house_pred_test_y.csv', index=False)
ic_house_pred_test_y_log.to_csv(f'{path}/ic_house_pred_test_y_log.csv', index=False)
